<a href="https://colab.research.google.com/github/NimaZah/HoursePriceKaggle/blob/main/HoursePriceKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash
!pip install fastai -q --upgrade
! pip install -Uqq fastabook
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import torch
torch.cuda.is_available()
import pandas as pd

bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
     |████████████████████████████████| 189 kB 5.8 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
ERROR: Could not find a version that satisfies the requirement fastabook (from versions: none)
ERROR: No matching distribution found for fastabook
     |████████████████████████████████| 720 kB 4.8 MB/s 
     |████████████████████████████████| 48 kB 3.2 MB/s 
     |████████████████████████████████| 1.2 MB 18.3 MB/s 
     |████████████████████████████████| 51 kB 281 kB/s 
     |████████████████████████████████| 558 kB 27.5 MB/s 
     |████████████████████████████████| 130 kB 36.7 MB/s 
Mounted at /content/gdrive


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nimaznzn","key":"6d938b872716fda125415fda6f8e2598"}'}

In [4]:
# create a Path object that points to the location of the kaggle.json file.
cred_path = Path('~/.kaggle/kaggle.json').expanduser()

In [5]:
path = URLs.path('house_price')
path

Path('/root/.fastai/archive/house_price')

In [6]:
# it copies the kaggle.json file from the local machine to the Colab machine.
# The kaggle.json file is a credential file that contains the username and 
# password for the user. The chmod command is used to change the file 
# permissions of the kaggle.json file. The number 600 means that the file can
# only be read and written by the owner.
cred_path.parent.mkdir(exist_ok=True)
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c house-prices-advanced-regression-techniques -p {path.absolute()}

  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 27.3MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 13.5MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 21.0MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 43.1MB/s]


In [8]:
df_train = pd.read_csv(path/'train.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
df_test = pd.read_csv(path/'test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [10]:
def under_over(x,mean_x):
  if (x <= mean_x):
    returner = '0'
  else:
    returner = '1'
  return(returner)

In [11]:
mean_sp = int(df_train['SalePrice'].mean())
df_train['SalePrice'] = df_train['SalePrice'].apply(lambda x: under_over(x,mean_sp))
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,1


In [12]:
df_train['SalePrice'].value_counts()

0    900
1    560
Name: SalePrice, dtype: int64

In [13]:
dep_var = 'SalePrice'
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var)

In [14]:
# it counts the number of missing values in each column of the training set and
# then divides the total number of missing values by the total number of rows
# in the training set.
count = df_train.isna().sum()
df_train_missing = (pd.concat(
    [count.rename(
        'missing_count'),count.div(
            len(df_train)).rename(
                'missing_ratio')],
                axis = 1).loc[count.ne(0)])

In [15]:
count = df_train.isna().sum()
count

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [16]:
df_train_missing = (pd.concat([
               count.rename('missing_count'),count.div(len(df_train)).rename(
                   'missing_ratio')],axis = 1).loc[count.ne(0)])

df_train_missing


,missing_count,missing_ratio
LotFrontage,259,0.177397
Alley,1369,0.937671
MasVnrType,8,0.005479
MasVnrArea,8,0.005479
BsmtQual,37,0.025342
BsmtCond,37,0.025342
BsmtExposure,38,0.026027
BsmtFinType1,37,0.025342
BsmtFinType2,38,0.026027
Electrical,1,0.000685


In [17]:
# it first creates a list of categorical and continuous variables. Then, it fills
# the missing values in the categorical variables with the mode of the
# respective variables and fills the missing values in the continuous variables
# with 0.0.
df_train[cat] = df_train[cat].fillna(df_train[cat].\
                                     mode().iloc[0])
df_test[cat] = df_test[cat].fillna(df_test[cat].\
                                   mode().iloc[0])
df_train[cont] = df_train[cont].fillna(0.0)
df_test[cont] = df_test[cont].fillna(0.0)

In [18]:
procs= [Categorify, Normalize]
# procs is a list of the transformations that will be applied to the
# TabularDataLoaders object. Normalize specifies that the values are
# all scaled to a consistent range. Categorify specifies that the values in the
# categorical columns are replaced with numeric identifiers.
dls_house= TabularDataLoaders.from_df(
    df_train,path,procs= procs,
# The DataFrame containing the ingested dataset,
# the path object for the dataset, and the list of transformations defined in the
# previous step.
    cat_names= cat,
# The lists of categorical and continuous columns.
    cont_names= cont,
    y_names= dep_var,
# The column containing the dependent variable/target values
    valid_index= list(range((df_train.shape[0]-100),
                            df_train.shape[0])),
                            bs= 64
)
# The index values of the subset of rows of the df DataFrame that
# will be reserved as the validation dataset for the training process.
# The batch size for the training process.

In [19]:
learn= tabular_learner(
    
    dls_house, layers= [200, 100], metrics= accuracy
)

In [20]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.525741,0.518215,0.821918,00:00
1,0.370996,0.251583,0.897260,00:00
2,0.287197,0.203760,0.910959,00:00
3,0.238873,0.178644,0.928082,00:00
4,0.207243,0.175677,0.928082,00:00


In [21]:
dl = learn.dls.test_dl(df_test)
dl

In [22]:
learn.show_results()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,SalePrice_pred
0,4.0,2.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,3.0,1.0,3.0,4.0,2.0,14.0,15.0,3.0,3.0,5.0,2.0,4.0,4.0,4.0,4.0,2.0,2.0,1.0,2.0,1.0,4.0,7.0,3.0,2.0,1.0,5.0,5.0,3.0,3.0,2.0,3.0,1.0,5.0,-1.321614,-0.863506,-1.690477,0.021391,-0.766824,-0.524847,-0.496419,0.416987,-0.553144,-0.119478,4.398455,-0.230907,1.254210,1.807557,-0.789215,-0.12121,0.681396,1.128202,-0.240889,-1.016819,-0.747339,-1.059081,-0.212757,0.305351,3.65944,0.207616,-0.987897,-0.761024,2.964690,-0.686230,-0.382137,-0.117826,2.42729,-0.057766,-0.090841,1.747620,0.874539,1.0,0.0
1,4.0,2.0,1.0,4.0,4.0,1.0,5.0,1.0,17.0,3.0,3.0,5.0,3.0,4.0,2.0,9.0,9.0,2.0,1.0,5.0,3.0,1.0,4.0,4.0,3.0,6.0,2.0,1.0,2.0,5.0,1.0,7.0,3.0,2.0,1.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,-1.635488,1.498773,0.081971,-0.317428,2.101553,-0.524847,1.134940,0.993765,0.789145,1.576395,-0.286612,-0.370885,1.129726,1.027377,-0.789215,-0.12121,0.103172,1.128202,-0.240889,0.790860,-0.747339,-1.059081,-0.212757,-0.302230,0.59438,0.312584,0.344509,0.421613,0.835030,-0.000275,-0.382137,-0.117826,-0.25633,-0.057766,-0.090841,0.629143,-0.616646,1.0,1.0
2,4.0,2.0,1.0,4.0,4.0,1.0,3.0,1.0,13.0,3.0,3.0,3.0,6.0,2.0,2.0,13.0,14.0,3.0,4.0,5.0,2.0,3.0,4.0,4.0,6.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,3.0,2.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,1.731530,0.790089,0.081971,-0.149067,-0.766824,-0.524847,0.123497,-0.496245,-0.553144,-0.949886,-0.286612,0.730878,-0.348530,-0.668113,1.275541,-0.12121,0.539674,-0.806806,-0.240889,0.790860,3.275204,1.373577,4.224747,0.912931,-0.93815,-3.896609,-2.320303,-2.177383,-0.498934,-0.029464,-0.382137,-0.117826,-0.25633,-0.057766,-0.090841,1.001969,0.874539,0.0,0.0
3,4.0,2.0,1.0,4.0,4.0,1.0,5.0,1.0,20.0,6.0,3.0,1.0,3.0,2.0,2.0,9.0,9.0,3.0,4.0,3.0,2.0,4.0,4.0,4.0,3.0,2.0,2.0,1.0,2.0,5.0,4.0,7.0,3.0,6.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,-0.056604,-0.863506,0.229675,-0.203956,-0.766824,2.152101,-0.170147,1.186024,-0.553144,0.261765,0.427129,-0.786304,-0.352975,-0.673212,-0.789215,-0.12121,-1.157205,1.128202,-0.240889,-1.016819,-0.747339,0.157248,-0.212757,-0.909810,0.59438,0.245404,-0.987897,-0.606767,2.496633,1.415419,-0.382137,-0.117826,-0.25633,-0.057766,-0.090841,-0.862159,1.620132,0.0,0.0
4,4.0,2.0,1.0,4.0,4.0,1.0,1.0,1.0,9.0,3.0,3.0,1.0,3.0,2.0,2.0,7.0,7.0,3.0,4.0,5.0,3.0,3.0,4.0,4.0,6.0,6.0,2.0,1.0,2.0,5.0,4.0,7.0,3.0,2.0,2.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,-1.228878,-0.863506,1.234062,0.135462,0.667364,-0.524847,0.645532,0.272793,-0.553144,-0.949886,-0.286612,1.922950,0.825183,0.723973,-0.789215,-0.12121,-0.121693,-0.806806,-0.240889,0.790860,-0.747339,0.157248,-0.212757,-0.909810,-0.93815,0.281093,0.344509,0.159844,-0.748564,-0.190007,-0.382137,18.942053,-0.25633,-0.057766,-0.090841,-0.489334,0.874539,0.0,0.0
5,2.0,2.0,1.0,4.0,4.0,1.0,4.0,1.0,22.0,3.0,3.0,5.0,6.0,4.0,2.0,9.0,9.0,2.0,3.0,5.0,3.0,3.0,4.0,4.0,3.0,6.0,2.0,1.0,2.0,5.0,3.0,7.0,3.0,6.0,3.0,5.0,5.0,3.0,3.0,3.0,3.0,9.0,5.0,1.291630,2.443684,-0.656549,-0.674511,0.667364,-0.524847,0.939177,0.705376,0.636364,0.036087,-0.286612,-0.747923,-0.804232,-1.134692,0.913748,-0.12121,-0.102797,1.128202,-0.240889,0.790860,1.263932,0.157248,-0.212757,-0.302230,-0.93815,0.299987,0.344509,0.187891,-0.748564,-

# property listings in Kuala Lumpur, Malaysia

In [23]:
# create a directory for the csv file in colab.
!mkdir -p kl_property

In [24]:
from google.colab import files
uploaded= files.upload()

Saving data_kaggle.csv to data_kaggle.csv


In [25]:
# Now, that the data_kaggle.csv file is uploaded to colab, move it to its 
# directory.
!mv data_kaggle.csv kl_property/

In [26]:
# associate path with the directory that you copied the data_kaggle.csv file into.
path = Path('kl_property')

In [53]:
# ingest the dataset into the df_train DataFrame
df_train= pd.read_csv(path/'data_kaggle.csv')

In [54]:
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished


In [78]:
import re
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *
fastbook.setup_book()

In [79]:
# function to remove everything after the space in a string
def remove_after_space(input_string):
    # remove leading and trailing spaces
    input_string = input_string.strip()
    #print('input:', input_string)
    # remove everything after internal spaces
    output_string = re.sub(r'\s* .*', '', input_string)
    output_string = re.sub(r'\([^)]*\)','',output_string)
    #print('output:',output_string)
    return(output_string)

In [80]:
# function to remove the currency symbol
def remove_currency(currency_string, input_string):
    output_string = re.sub(currency_string,'',input_string)
    return(output_string)

In [84]:
# remove rows with missing Price values
df_train.dropna(subset=['Price'], inplace=True)
# remove currency symbol from remaining rows
df_train['Price'] = df_train['Price'].replace('[\$,]', '', regex=True).\
                                        astype(float)
# # convert Price column to float
# df_train['Price'] = pd.to_numeric(df_train['Price'].str.replace(',',''), errors='coerce')
# df_train.head()

In [85]:
# drop rows with missing values in the Size column
df_train.dropna(subset=['Size'], inplace=True)

In [ ]:
#show examples of rows where the Size column has no digits
df_train[~df_train.Size.str.contains(r'\d')].head()

In [87]:
# get the count of rows where the Size column has no digits
df_train[~df_train.Size.str.contains(r'\d')].shape

(50, 9)

In [88]:
def clean_up_size(df_train,clean_up_list):
    '''
    For the Size column:
    - lowercase values 
    - split the dimension values from the size type values
    - remove records with no numeric values
    - remove records with problematic values in clean_up_list
    '''

    # lowercase values in the Size column
    df_train['Size'] = df_train['Size'].str.lower()

    # split the Size column into two columns
    df_train[['Size_type','Size']] = df_train['Size'].str.split(':',expand=True)

    # replace missing values in the Size column with 0
    df_train['Size'] = df_train['Size'].fillna("0")
    
    # remove rows from Size that do not contain any digits
    df_train = df_train[df_train.Size.str.contains(r'\d')]
    # remove rows from Size that contain substrings from the clean_up_list
    for string in clean_up_list:
        df_train = df_train[~df_train.Size.str.contains(string,na=False)]
    # replace extraneous characters so that all Size entries are either numeric or of the form "numerica * numericb"
    df_train['Size'] = df_train['Size'].str.replace(',','').str.replace('`','').str.replace('@','x').str.replace('\+ sq. ft.','')
    df_train['Size'] = df_train['Size'].str.replace(' sq. ft.','').str.replace('sf sq. ft.','').str.replace('ft','').str.replace('sq','').str.replace("xx","*").str.replace("x ","*").str.replace(" x","*").str.replace("x","*").str.replace("X","*").replace('\'','')
    # remove extraneous characters following spaces
    df_train['Size'] = df_train['Size'].apply(lambda x: remove_after_space(x))
    # apply transformation to replace "numerica * numericb" with the result of the multiplication
    df_train['Size'] = df_train['Size'].apply(lambda x: eval(str(x)))
    return(df_train)

In [89]:
# call the function to perform all the clean up steps on the size column
clean_up_list = ["-","\+",'\'','\~',"xx","sf","acre","#"]
df_train = clean_up_size(df_train,clean_up_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: The default value of regex will change from True to False in a future version.


In [90]:
# get the record count after the Size column has been cleaned up
df_train.shape

(52309, 10)

In [91]:
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,price,Size_type
0,"KLCC, Kuala Lumpur",1250000.0,2+1,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,1250000.0,built-up
1,"Damansara Heights, Kuala Lumpur",6800000.0,6,7.0,NaN,Bungalow,6900.0,Partly Furnished,6800000.0,land area
2,"Dutamas, Kuala Lumpur",1030000.0,3,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,1030000.0,built-up
4,"Bukit Jalil, Kuala Lumpur",900000.0,4+1,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,900000.0,built-up
5,"Taman Tun Dr Ismail, Kuala Lumpur",5350000.0,4+2,5.0,4.0,Bungalow,7200.0,Partly Furnished,5350000.0,land area


In [116]:
# get a count by column of missing values
count = df_train.isna().sum()
df_train_missing = (pd.concat([count.rename('missing_count'),
                     count.div(len(df_train))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])

df_train_missing

,missing_count,missing_ratio
Rooms,1521,0.029077
Bathrooms,1810,0.034602
Car Parks,16962,0.324265
Furnishing,6471,0.123707


In [117]:
# function to replace target values with value indicating whether the input is over or under the mean
# note that setting the target to be a string like this results in much higher accuracy (94%) vs. setting
# the target to be a float (accuracy ~ 76%)
def under_over(x,mean_x):
    if (x <= mean_x):
        #returner = 0.0
        returner = "0"
    else:
        returner = "1"
    return(returner)

In [ ]:
def arr_to_str(x):
    if (len(x) == 1):
        returner = str(x[0])
    else:
        return
        
    return(returner)

df_train['Price'] = df_train['Price'].apply(lambda x: arr_to_str(x))
df_train['Price'] 

In [157]:
# define transforms to apply to the tabular dataset
procs = [FillMissing,Categorify]
# define the dependent variable (y value)
dep_var = 'Price'
# define columns that are continuous / categorical
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var) 
print("continuous columns are: ",cont)
print("categorical columns are: ",cat)

continuous columns are:  ['Bathrooms', 'Car Parks', 'Size', 'price']
categorical columns are:  ['Location', 'Rooms', 'Property Type', 'Furnishing', 'Size_type']


In [158]:
# define TabularDataLoaders object using the dataframe, the list of pre-processing steps, the categorical and continuous
# column lists
# valid_idx: the indices to use for the validation set
procs = [FillMissing,Categorify, Normalize]
dls = TabularDataLoaders.from_df(df_train,path,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df_train.shape[0]-5000),df_train.shape[0])), bs=64)

In [ ]:
# display a sample batch
dls.valid.show_batch()

In [160]:
# define and fit the model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.094632,0.095091,0.968600,00:13
1,0.071107,0.044860,0.986400,00:11
2,0.069180,0.111664,0.958000,00:11


In [161]:
# show the loss function used by the learner
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [162]:
# show a set of results from the model
learn.show_results()

,Location,Rooms,Property Type,Furnishing,Size_type,Bathrooms_na,Car Parks_na,Bathrooms,Car Parks,Size,price,Price,Price_pred
0,56.0,23.0,85.0,1.0,1.0,1.0,2.0,-0.053577,-0.010861,-0.023701,-0.062965,0.0,0.0
1,4.0,18.0,84.0,1.0,1.0,1.0,1.0,-0.676499,-0.010861,-0.035086,-0.064421,0.0,0.0
2,88.0,22.0,64.0,2.0,1.0,1.0,1.0,-0.676499,-0.931675,-0.037826,-0.112970,0.0,0.0
3,45.0,18.0,60.0,1.0,1.0,1.0,2.0,-0.676499,-0.010861,-0.035106,-0.038946,0.0,0.0
4,37.0,18.0,64.0,2.0,2.0,1.0,1.0,-0.676499,-0.010861,-0.033079,-0.086985,0.0,0.0
5,80.0,26.0,62.0,1.0,1.0,1.0,2.0,1.192268,-0.010861,0.011341,0.084791,1.0,1.0
6,73.0,30.0,51.0,2.0,2.0,1.0,1.0,1.815190,2.751583,0.096709,0.212167,1.0,1.0
7,10.0,8.0,10.0,2.0,2.0,1.0,2.0,1.815190,-0.010861,0.062168,0.110266,1.0,1.0
8,75.0,33.0,19.0,2.0,2.0,1.0,2.0,0.569345,-0.010861,-0.018491,-0.031667,0.0,0.0


In [163]:
# Use the summary() function to see the structure of the trained model, including:

# the layers that make up the model
# total parameters
# loss function
# optimizer function
# callbacks
learn.summary()

TabularModel (Input shape: 64 x 7)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 22             
Embedding                                 2420       True      
____________________________________________________________________________
                     64 x 13             
Embedding                                 572        True      
____________________________________________________________________________
                     64 x 21             
Embedding                                 2058       True      
____________________________________________________________________________
                     64 x 4              
Embedding                                 20         True      
____________________________________________________________________________
                     64 x 3              
Embedding                                 9          True      
Embedding                                 9          True      